# Pipeline de ETL: Camada Bronze para Prata

**Autor:** Diego Carlito Rodrigues de Souza
**Matrícula:** 221007690

**Objetivo:** Construir um pipeline de ETL utilizando PySpark para ler os dados brutos da camada Bronze, aplicar as regras de negócio de limpeza e transformação definidas na modelagem, e gerar a tabela consolidada `pedidos` na camada Prata.

In [1]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType

spark = SparkSession.builder \
    .appName("ETLBronzeParaPrataOlist") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/11 08:09:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 1. Extração (Extract)
Nesta etapa, carregamos todos os datasets da camada Bronze para DataFrames PySpark.

In [2]:
caminho_base = "../data/bronze/"

dfs = {}

arquivos = [
    "olist_customers_dataset.csv", "olist_order_items_dataset.csv",
    "olist_order_payments_dataset.csv", "olist_order_reviews_dataset.csv",
    "olist_orders_dataset.csv", "olist_products_dataset.csv",
    "olist_sellers_dataset.csv", "product_category_name_translation.csv"
]

for arquivo in arquivos:
    nome_chave = arquivo.replace("_dataset.csv", "").replace(".csv", "")
    dfs[nome_chave] = spark.read.csv(f"{caminho_base}{arquivo}", header=True, inferSchema=True)

print("DataFrames carregados da camada Bronze:")
for nome in dfs.keys():
    print(f"- {nome}")

DataFrames carregados da camada Bronze:
- olist_customers
- olist_order_items
- olist_order_payments
- olist_order_reviews
- olist_orders
- olist_products
- olist_sellers
- product_category_name_translation


## 2. Transformação (Transform)
Esta é a etapa principal do ETL, onde aplicamos todas as regras de negócio:
1.  Unificar os DataFrames através de `JOINs`.
2.  Limpar e tratar dados (nulos, tipos de dados).
3.  Criar novas colunas (engenharia de features).
4.  Renomear e selecionar as colunas finais.

In [3]:
df_prata = dfs["olist_order_items"].join(
    dfs["olist_orders"], "order_id", "left"
).join(
    dfs["olist_products"], "product_id", "left"
).join(
    dfs["olist_sellers"], "seller_id", "left"
).join(
    dfs["olist_customers"], "customer_id", "left"
).join(
    dfs["olist_order_payments"], "order_id", "left"
).join(
    dfs["olist_order_reviews"], "order_id", "left"
).join(
    dfs["product_category_name_translation"], "product_category_name", "left"
)

print("JOINs concluídos. Schema após unificação:")
df_prata.printSchema()

JOINs concluídos. Schema após unificação:
root
 |-- product_category_name: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = t

In [4]:
df_prata_transformado = df_prata.select(
    # --- CHAVES ---
    F.concat(F.col("order_id"), F.lit("_"), F.col("order_item_id")).alias("id_item_pedido"),
    F.col("order_id").alias("id_pedido"),
    F.col("customer_unique_id").alias("id_cliente"),
    F.col("product_id").alias("id_produto"),
    F.col("seller_id").alias("id_vendedor"),
    
    # --- DADOS DO PEDIDO ---
    F.col("order_status").alias("status_pedido"),
    F.col("order_purchase_timestamp").alias("data_compra"),
    F.col("order_approved_at").alias("data_aprovacao"),
    F.col("order_delivered_carrier_date").alias("data_envio_transportadora"),
    F.col("order_delivered_customer_date").alias("data_entrega_cliente"),
    F.col("order_estimated_delivery_date").alias("data_estimada_entrega"),

    # --- DADOS DO ITEM ---
    F.col("price").cast("decimal(10,2)").alias("preco_item"),
    F.col("freight_value").cast("decimal(10,2)").alias("valor_frete"),
    F.coalesce(F.col("product_category_name_english"), F.lit("desconhecida")).alias("categoria_produto"),

    # --- DADOS DO CLIENTE E VENDEDOR ---
    F.col("customer_city").alias("cidade_cliente"),
    F.col("customer_state").alias("estado_cliente"),
    F.col("seller_city").alias("cidade_vendedor"),
    F.col("seller_state").alias("estado_vendedor"),

    # --- DADOS DE PAGAMENTO ---
    F.col("payment_type").alias("tipo_pagamento"),
    F.col("payment_installments").alias("parcelas_pagamento"),
    F.col("payment_value").cast("decimal(10,2)").alias("valor_pagamento"),

    # --- DADOS DE AVALIAÇÃO (com limpeza) ---
    F.expr("try_cast(review_score as int)").alias("nota_avaliacao"),
    
    # --- COLUNAS CALCULADAS (ENGENHARIA DE FEATURES) ---
    F.datediff(F.col("order_delivered_customer_date"), F.col("order_purchase_timestamp")).alias("tempo_entrega_dias"),
    (F.col("order_delivered_customer_date") > F.col("order_estimated_delivery_date")).alias("atraso_na_entrega")
)

df_prata_final = df_prata_transformado.filter(
    (F.col("nota_avaliacao").isNull()) | (F.col("nota_avaliacao").between(1, 5))
)

print("\nTransformações concluídas. Schema final da camada Prata:")
df_prata_final.printSchema()

print("\nAmostra dos dados transformados:")
df_prata_final.show(5)


Transformações concluídas. Schema final da camada Prata:
root
 |-- id_item_pedido: string (nullable = true)
 |-- id_pedido: string (nullable = true)
 |-- id_cliente: string (nullable = true)
 |-- id_produto: string (nullable = true)
 |-- id_vendedor: string (nullable = true)
 |-- status_pedido: string (nullable = true)
 |-- data_compra: timestamp (nullable = true)
 |-- data_aprovacao: timestamp (nullable = true)
 |-- data_envio_transportadora: timestamp (nullable = true)
 |-- data_entrega_cliente: timestamp (nullable = true)
 |-- data_estimada_entrega: timestamp (nullable = true)
 |-- preco_item: decimal(10,2) (nullable = true)
 |-- valor_frete: decimal(10,2) (nullable = true)
 |-- categoria_produto: string (nullable = false)
 |-- cidade_cliente: string (nullable = true)
 |-- estado_cliente: string (nullable = true)
 |-- cidade_vendedor: string (nullable = true)
 |-- estado_vendedor: string (nullable = true)
 |-- tipo_pagamento: string (nullable = true)
 |-- parcelas_pagamento: intege

+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-------------------+-------------------+-------------------------+--------------------+---------------------+----------+-----------+-----------------+--------------------+--------------+---------------+---------------+--------------+------------------+---------------+--------------+------------------+-----------------+
|      id_item_pedido|           id_pedido|          id_cliente|          id_produto|         id_vendedor|status_pedido|        data_compra|     data_aprovacao|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|preco_item|valor_frete|categoria_produto|      cidade_cliente|estado_cliente|cidade_vendedor|estado_vendedor|tipo_pagamento|parcelas_pagamento|valor_pagamento|nota_avaliacao|tempo_entrega_dias|atraso_na_entrega|
+--------------------+--------------------+--------------------+--------------------+--------------------+------------

## 3. Carga (Load)
Nesta etapa final, o DataFrame transformado é salvo na camada Prata. Escolhemos o formato **Parquet** por ser otimizado para performance em ambientes de Big Data, oferecendo compressão colunar e melhor integração com o ecossistema Spark.

In [5]:
caminho_saida = "../data/silver/pedidos"

df_prata_final.write.mode("overwrite").parquet(caminho_saida)

print(f"DataFrame salvo com sucesso em formato Parquet no caminho: {caminho_saida}")

25/10/11 08:09:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


DataFrame salvo com sucesso em formato Parquet no caminho: ../data/silver/pedidos


In [6]:
spark.stop()